In [ ]:
import requests
import json
from http import client
from collections import OrderedDict

from google.cloud import bigquery
from google.cloud.bigquery import SchemaField

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./gtm-ops-9e9a928ef0ac.json"
client = bigquery.Client()

In [ ]:
# Define the table to write to
table_id_staging = "gtm-ops.hubspot_deals.staging_marketing_table"
table_id_main = "gtm-ops.hubspot_deals.marketing_table"
